In [3]:
#pip install scikit-learn
#pip install faiss-cpu

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
#https://github.com/facebookresearch/faiss/blob/main/README.md
import numpy as np


In [4]:
jobs_final=pd.read_pickle('/Users/sodagayethiam/Documents/Formations/Parcours_data_engineer/projet_jobmarket/JobMarket/outputs/final/jobs.pkl')
jobs_final.columns
jobs_final = jobs_final.loc[jobs_final['lang'] == 'en']


Index(['job_title', 'contract_time', 'company_name', 'salary_min',
       'salary_max', 'publication_datetime', 'location', 'industry',
       'offer_link', 'source', 'levels', 'category', 'job_description',
       'publication_time', 'publication_date', 'contract_type', 'country',
       'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name',
       'country_code', 'codeIso_lvl4', 'lang', 'Quality_of_Life_Index',
       'Purchasing_Power_Index', 'Safety_Index', 'Health_Care_Index',
       'Cost_of_Living_Index', 'Rent_Index', 'Groceries_Index', 'Crime_Index',
       'Climate_Index', 'Pollution_Index', 'Traffic_Index',
       'Affordability_Index'],
      dtype='object')

In [31]:
jobs_final.contract_type.value_counts()

contract_type
not available                             78248
permanent                                 32086
contract                                   9857
external                                   1937
fixed-term / temporary                      977
                                          ...  
fixed-term / temporary(10 months)             1
other(3 to 6 months)                          1
internship(4 to 9 months)                     1
work study(10 to 24 months)                   1
fixed-term / temporary(1 to 10 months)        1
Name: count, Length: 138, dtype: int64

In [32]:
#encode categorical columns
ohe = OneHotEncoder( drop="first", sparse_output=False)
cat = ['contract_time', 'contract_type', 'industry', 'category', 'levels']
jobs_encoded = ohe.fit_transform(jobs_final[cat])

encoded_df = pd.DataFrame(jobs_encoded, columns=ohe.get_feature_names_out(cat))
#(125214, 837)

In [33]:
num_col=['salary_min', 'salary_max']#, 'Quality_of_Life_Index', 'Purchasing_Power_Index', 'Safety_Index', 'Health_Care_Index',
#       'Cost_of_Living_Index', 'Rent_Index', 'Groceries_Index', 'Crime_Index','Climate_Index', 'Pollution_Index', 'Traffic_Index', 'Affordability_Index']

scaler = MinMaxScaler()
jobs_num = scaler.fit_transform(jobs_final[num_col])
jobs_num_df = pd.DataFrame(jobs_num, columns=num_col)
jobs_num_df

,salary_min,salary_max
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
125209,0.0,0.0
125210,0.0,0.0
125211,0.0,0.0
125212,0.0,0.0


In [35]:

vectorizer = TfidfVectorizer(max_features=100)
job_titles_tfidf = vectorizer.fit_transform(jobs_final['job_title'])
job_desc_tfidf = vectorizer.fit_transform(jobs_final['job_description'])

In [36]:
job_titles_tfidf = pd.DataFrame(job_titles_tfidf.toarray(), 
                                 columns=vectorizer.get_feature_names_out(), 
                                 index=jobs_final.index)
job_desc_tfidf = pd.DataFrame(job_desc_tfidf.toarray(), 
                                 columns=vectorizer.get_feature_names_out(), 
                                 index=jobs_final.index)
jobs_reco = pd.concat([encoded_df, jobs_num_df], axis=1)
jobs_reco = pd.concat([jobs_reco, job_titles_tfidf], axis=1)
jobs_reco = pd.concat([jobs_reco, job_desc_tfidf], axis=1)
jobs_reco.shape #(125214, 1051)

# convert  float32 for Faiss input and with continguous format for the cosinus distance
#jobs_reco_np = jobs_reco.to_numpy().astype('float32')
jobs_reco_np = np.ascontiguousarray(jobs_reco.to_numpy().astype('float32'))

#normalise for cosinus distance 
faiss.normalize_L2(jobs_reco_np)
index = faiss.IndexFlatIP(jobs_reco_np.shape[1])  # IP = Inner Product, équivalent à cosinus avec données normalisées
index.add(jobs_reco_np)  # Ajout des données

# Searchng for k nearest neighborhood for each job  
k=10
distances, indices = index.search(jobs_reco_np, k)


In [38]:
k

10

In [67]:
input_job_title = "Data engineer"

job_index = jobs_final[jobs_final['job_title'].str.contains(input_job_title, case=False)].index[0]
neighbor_indices = indices[job_index]

recommended_jobs = jobs_final.iloc[neighbor_indices]    


In [68]:
recommended_jobs

,job_title,contract_time,company_name,salary_min,salary_max,publication_datetime,location,industry,offer_link,source,...,Safety_Index,Health_Care_Index,Cost_of_Living_Index,Rent_Index,Groceries_Index,Crime_Index,Climate_Index,Pollution_Index,Traffic_Index,Affordability_Index
185,(Senior) Data Engineer,full_time,MIRAKL,0.0,0.0,2024-08-08T07:43:48Z,Paris,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,42.300000,77.300000,73.6,40.5,80.1,57.700000,88.400000,64.500000,152.500000,0.80
184,(Senior) Data Engineer,full_time,MIRAKL,0.0,0.0,2024-08-08T07:43:45Z,Paris,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,42.300000,77.300000,73.6,40.5,80.1,57.700000,88.400000,64.500000,152.500000,0.80
183,(Senior) Data Engineer,full_time,MIRAKL,0.0,0.0,2024-08-08T07:43:51Z,Bordeaux,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,51.914022,70.820699,29.4,8.3,28.6,48.085978,86.955062,46.584288,159.092658,0.90
100237,Senior Data Engineer (H/F/N) - Paris,full_time,EKIMETRICS,0.0,0.0,2024-05-16T00:58:43Z,Paris,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,42.300000,77.300000,73.6,40.5,80.1,57.700000,88.400000,64.500000,152.500000,0.80
25190,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:09Z,Toulouse,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,50.400000,77.300000,73.6,40.5,80.1,49.600000,88.400000,62.566667,152.500000,1.15
25189,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:12Z,Nantes,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,45.263636,77.300000,73.6,40.5,80.1,54.736364,88.400000,62.566667,152.500000,1.15
25188,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:11Z,Lyon,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,44.500000,77.300000,73.6,40.5,80.1,55.500000,88.400000,51.600000,152.500000,1.50
25187,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:08Z,Lille,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,49.100000,77.300000,73.6,40.5,80.1,50.900000,88.400000,62.566667,152.500000,1.15
25186,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:06Z,Bordeaux,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,50.500000,77.300000,73.6,40.5,80.1,49.500000,88.400000,62.566667,152.500000,1.15
9250,Architecte Data - Senior - H/F - CDI,full_time,TALAN,0.0,0.0,2024-07-08T15:53:07Z,Lyon,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,44.500000,77.300000,73.6,40.5,80.1,55.500000,88.400000,51.600000,152.500000,1.50


array([1, 7, 9, 8, 5, 3, 6, 4, 0, 2])

In [60]:
"""from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(jobs_reco)
The Kernel crashed while executing code in the the current cell or a previous cell. Please review the code in the cell(s) to identify a possible cause of the failure. Click here for more info. View Jupyter log for further details.
"""

def weighted_recommendations(input_job_title,k=10):
    job_index = jobs_final[jobs_final['job_title'].str.contains(input_job_title, case=False)].index[0]
    neighbor_indices = indices[job_index]

    recommended_jobs = jobs_final.iloc[neighbor_indices]    
    indices_for_weighting = [
        'Quality_of_Life_Index', 'Purchasing_Power_Index','Safety_Index','Health_Care_Index', 'Cost_of_Living_Index', 'Rent_Index', 
        'Groceries_Index', 'Crime_Index', 'Climate_Index','Pollution_Index','Traffic_Index', 'Affordability_Index']
    
    indices_values = recommended_jobs[indices_for_weighting].values
    
    # Invert the index for Cost_of_Living_Index , Crime_Index,Pollution_Index,Traffic_Index,Rent_Index,
    indices_values[:,4 ] = 1 / (indices_values[:,4 ] + 0.1)  # Cost_of_Living_Index
    indices_values[:,5 ] = 1 / (indices_values[:,5 ] + 0.1)  #  Rent_Index
    indices_values[:,7 ] = 1 / (indices_values[:,7 ] + 0.1)  #  Crime_Index
    indices_values[:,9 ] = 1 / (indices_values[:,9 ] + 0.1)  #  Pollution_Index
    indices_values[:,10 ] = 1 / (indices_values[:,10 ] + 0.1)  #  Traffic_Index

    weights = np.sum(indices_values, axis=1)
    # normalize weights 
    weights = (weights - np.min(weights)) / (np.max(weights) - np.min(weights))
    weighted_distances = neighbor_distances * weights
    weighted_indices = np.argsort(weighted_distances)[:k]
    
    return recommended_jobs.iloc[weighted_indices]



In [66]:
recommended_jobs_w = weighted_recommendations(" data engineer")
recommended_jobs_w

,job_title,contract_time,company_name,salary_min,salary_max,publication_datetime,location,industry,offer_link,source,...,Safety_Index,Health_Care_Index,Cost_of_Living_Index,Rent_Index,Groceries_Index,Crime_Index,Climate_Index,Pollution_Index,Traffic_Index,Affordability_Index
183,(Senior) Data Engineer,full_time,MIRAKL,0.0,0.0,2024-08-08T07:43:51Z,Bordeaux,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,51.914022,70.820699,29.4,8.3,28.6,48.085978,86.955062,46.584288,159.092658,0.90
100237,Senior Data Engineer (H/F/N) - Paris,full_time,EKIMETRICS,0.0,0.0,2024-05-16T00:58:43Z,Paris,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,42.300000,77.300000,73.6,40.5,80.1,57.700000,88.400000,64.500000,152.500000,0.80
184,(Senior) Data Engineer,full_time,MIRAKL,0.0,0.0,2024-08-08T07:43:45Z,Paris,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,42.300000,77.300000,73.6,40.5,80.1,57.700000,88.400000,64.500000,152.500000,0.80
185,(Senior) Data Engineer,full_time,MIRAKL,0.0,0.0,2024-08-08T07:43:48Z,Paris,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,42.300000,77.300000,73.6,40.5,80.1,57.700000,88.400000,64.500000,152.500000,0.80
9250,Architecte Data - Senior - H/F - CDI,full_time,TALAN,0.0,0.0,2024-07-08T15:53:07Z,Lyon,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,44.500000,77.300000,73.6,40.5,80.1,55.500000,88.400000,51.600000,152.500000,1.50
25188,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:11Z,Lyon,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,44.500000,77.300000,73.6,40.5,80.1,55.500000,88.400000,51.600000,152.500000,1.50
25189,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:12Z,Nantes,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,45.263636,77.300000,73.6,40.5,80.1,54.736364,88.400000,62.566667,152.500000,1.15
25187,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:08Z,Lille,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,49.100000,77.300000,73.6,40.5,80.1,50.900000,88.400000,62.566667,152.500000,1.15
25186,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:06Z,Bordeaux,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,50.500000,77.300000,73.6,40.5,80.1,49.500000,88.400000,62.566667,152.500000,1.15
25190,Cloud Data Engineer France H/F (CDI - Confirmé),full_time,TALAN,0.0,0.0,2024-07-08T15:53:09Z,Toulouse,not available,https://www.welcometothejungle.com/en/companie...,Wttj,...,50.400000,77.300000,73.6,40.5,80.1,49.600000,88.400000,62.566667,152.500000,1.15


In [16]:
"""#print(jobs_final.iloc[indices[10]])
job_indices = indices[10]  # Indices des jobs similaires
recommended_jobs = jobs_final.iloc[job_indices]
print(recommended_jobs)"""


'#print(jobs_final.iloc[indices[10]])\njob_indices = indices[10]  # Indices des jobs similaires\nrecommended_jobs = jobs_final.iloc[job_indices]\nprint(recommended_jobs)'

In [29]:
input_job_title = "Data scientist"

job_index = jobs_final[jobs_final['job_title'].str.contains(input_job_title, case=False)].index[0]
neighbor_indices = indices[job_index]

# Récupérer les jobs recommandés
recommended_jobs = jobs_final.iloc[neighbor_indices]
recommended_jobs

#je veux ordonner par qualite de vie 

,job_title,contract_time,company_name,salary_min,salary_max,publication_datetime,location,industry,offer_link,source,...,Safety_Index,Health_Care_Index,Cost_of_Living_Index,Rent_Index,Groceries_Index,Crime_Index,Climate_Index,Pollution_Index,Traffic_Index,Affordability_Index
12450,Astria - Data Scientist - Large Language Models,full_time,ASTRIA DIGITAL PVT.LTD,0.0,0.0,2024-10-09T07:07:32Z,"Bangalore, Karnataka",IT Jobs,https://www.adzuna.in/land/ad/4894731390?se=Zm...,adzuna,...,45.600000,65.100000,25.8,8.9,29.4,54.400000,89.400000,82.900000,252.900000,1.4
32794,Data Scientist - Statistical Modeling,full_time,CA-ONE INDIA,0.0,0.0,2024-11-08T10:14:25Z,"Bangalore, Karnataka",IT Jobs,https://www.adzuna.in/land/ad/4930829771?se=QJ...,adzuna,...,45.600000,65.100000,25.8,8.9,29.4,54.400000,89.400000,82.900000,252.900000,1.4
48641,Gallagher - Data Scientist - R/Python,full_time,GALLAGHER SERVICE CENTER LLP,0.0,0.0,2024-10-12T09:33:12Z,"Bangalore, Karnataka",IT Jobs,https://www.adzuna.in/land/ad/4899465750?se=Cj...,adzuna,...,45.600000,65.100000,25.8,8.9,29.4,54.400000,89.400000,82.900000,252.900000,1.4
87034,Publicis Sapient - Senior Data Engineer - Big ...,full_time,PUBLICIS SAPIENT,0.0,0.0,2024-11-12T10:17:55Z,"Bangalore, Karnataka",IT Jobs,https://www.adzuna.in/land/ad/4935651824?se=Zm...,adzuna,...,45.600000,65.100000,25.8,8.9,29.4,54.400000,89.400000,82.900000,252.900000,1.4
32449,Data Engineer - ETL/Azure Data Factory,full_time,ELEMENT INFOMATICS PVT. LTD.,0.0,0.0,2024-10-15T09:36:24Z,"Bangalore, Karnataka",IT Jobs,https://www.adzuna.in/land/ad/4902196775?se=AE...,adzuna,...,45.600000,65.100000,25.8,8.9,29.4,54.400000,89.400000,82.900000,252.900000,1.4
32787,Data Scientist - Predictive Modeling,full_time,TVARIT,0.0,0.0,2024-10-17T09:39:46Z,"Pune, Maharashtra",IT Jobs,https://www.adzuna.in/land/ad/4904818386?se=kj...,adzuna,...,59.000000,65.700000,24.1,6.6,25.5,41.000000,74.300000,71.300000,191.100000,1.5
14491,Azure Data Engineer - Data Factory/Databricks,full_time,ASCENT FORCE,0.0,0.0,2024-11-14T10:22:21Z,India,IT Jobs,https://www.adzuna.in/land/ad/4938341817?se=io...,adzuna,...,57.392308,65.123077,29.4,8.3,28.6,42.607692,65.655556,72.764865,215.755556,0.9
75976,Oracle Data Integration Engineer,full_time,NISH TECHNOLOGIES,0.0,0.0,2024-10-24T09:48:28Z,"Hyderabad, Telangana",IT Jobs,https://www.adzuna.in/land/ad/4911944804?se=Ur...,adzuna,...,56.800000,65.300000,22.8,6.4,25.6,43.200000,73.500000,76.100000,179.100000,1.8
32470,Data Engineer - Visualization Tools,full_time,VARITE INDIA,0.0,0.0,2024-11-16T06:31:39Z,"Bangalore, Karnataka",IT Jobs,https://www.adzuna.in/land/ad/4940844623?se=9E...,adzuna,...,45.600000,65.100000,25.8,8.9,29.4,54.400000,89.400000,82.900000,252.900000,1.4
32459,Data Engineer - Machine Learning,full_time,FLIPPED.AI,0.0,0.0,2024-11-15T06:29:49Z,"Bangalore, Karnataka",IT Jobs,https://www.adzuna.in/land/ad/4939518456?se=gN...,adzuna,...,45.600000,65.100000,25.8,8.9,29.4,54.400000,89.400000,82.900000,252.900000,1.4


In [20]:
"""pip install requests
#pip install Flask
pip install flask_pydantic
pip install pydantic
pip install fastapi"""

SyntaxError: invalid syntax (3650354277.py, line 2)

In [27]:
indices[5]

array([     5,  32364,  62453, 112390,  40747,  14957,  55723,  64770,
        19079, 106850])